# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [21]:
!cat publications.tsv





2022-01-01	f0	Prioritizing School Finance Equity during an Economic Downturn: Recommendations for State Policy Makers	Education Finance & Policy	"Abstract: State budgets temporarily crashed amid the COVID-19 pandemic and economic shutdown, placing education funding at risk. To demonstrate implications for school finance, we show that (1) school districts are racially segregated along class lines; (2) higher-poverty districts receive a greater share of funds from state, as opposed to local sources, making them especially vulnerable during economic downturns; and (3) many states made across-the-board K–12 budget reductions following the Great Recession, but those cuts disproportionately impacted high-poverty districts. A decade later, state legislators may face similar fiscal challenges. Instead of enacting across-the-board cuts, states can identify specific funding programs that already benefit lower-poverty districts or wealthier students. We demonstrate how this approach would wor

## Import pandas

We are using the very handy pandas library for dataframes.

In [22]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [23]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,forthcoming,title,venue,excerpt,citation,url_slug,paper_url
0,2015-08-18,f0,Persistence of Regional Wage Differences in China,Pacific Economic Review,Abstract: Regional wage differences in China a...,"Candelaria, C. A., Daly, M. & Hale, G. (2015)....",regional-wage-differences-china,https://doi.org/10.1111/1468-0106.12113
1,2017-08-30,f0,The Effects of No Child Left Behind on Childre...,AERA Open,Abstract: Many people have worried about possi...,"Whitney, C. R., & Candelaria, C. A. (2017). Th...",nclb-socioemotional-outcomes,https://doi.org/10.1177/2332858417726324
2,9999-01-01,f1,Spending More on the Poor? A Comprehensive Sum...,Education Finance & Policy,Abstract: Sixty-seven school finance reforms (...,"Shores, K. A., Candelaria, C. A., & Kabourek, ...",spending-more-on-the-poor,https://doi.org/10.1162/edfp_a_00360
3,2022-01-01,f0,Prioritizing School Finance Equity during an E...,Education Finance & Policy,Abstract: State budgets temporarily crashed am...,"Knight, D. S., Hassairi, N., Candelaria, C. A....",prioritizing-school-finance-equity,https://doi.org/10.1162/edfp_a_00356


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [24]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [25]:
import os
for row, item in publications.iterrows():
    
    if str(item.forthcoming) == "f0":
        md_filename =   str(item.pub_date) + "-" + item.url_slug + ".md"
        html_filename = str(item.pub_date) + "-" + item.url_slug
    else:
        md_filename =   str(item.pub_date) + "-" + "templink-forthcoming" + "-" + item.url_slug + ".md"
        html_filename = str(item.pub_date) + "-" + "templink-forthcoming" + "-" + item.url_slug
        
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += "\nforthcoming: " + item.forthcoming
    
    md += """\npermalink: /publications/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    ##if len(str(item.paper_url)) > 5:
    ##    md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    ##md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [26]:
!ls ../_publications/

2015-08-18-regional-wage-differences-china.md
2017-08-30-nclb-socioemotional-outcomes.md
2022-01-01-prioritizing-school-finance-equity.md
9999-01-01-spending-more-on-the-poor.md
9999-01-01-templink-forthcoming-spending-more-on-the-poor.md
9999-01-01-templink-forthcomingspending-more-on-the-poor.md


In [91]:
!cat ../_publications/2015-08-18-regional-wage-differences-china.md

---
title: "Persistence of Regional Wage Differences in China"
collection: publications
forthcoming: f0
permalink: /publications/2015-08-18-regional-wage-differences-china
excerpt: 'Abstract: Regional wage differences in China appear to be persistent and even to have grown over the past two decades. We study potential explanations for this phenomenon. After adjusting for the difference in the cost of living across provinces, we find that some of the cross-province differences in real wages could be related to the quality of labour, industry composition and geographic location of provinces. These factors, taken together, explain approximately half of the cross-province real wage variation. Interestingly, we find that interprovincial government transfers have not offset regional wage differences during the time period we consider. We also demonstrate that interprovincial migration, while driven in part by levels and changes in wage differences across provinces, did not help offset these 